# 🚀 Install, Import, and Log in

### Step 0️⃣: Install W&B

In [1]:
%%capture
#!pip install wandb

### Step 1️⃣: Import W&B and Login

In [2]:
import tensorflow as tf
from tensorflow import keras

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.feather as feather
from sklearn.model_selection import train_test_split 
from tensorflow.keras import regularizers
import math

%matplotlib inline

In [3]:
import wandb
from wandb.keras import WandbCallback


!wandb.login()

zsh: parse error near `()'


Read data in

In [ ]:
fluxData_df = feather.read_feather('data/fluxData.feather')
zernikeData_df = feather.read_feather('data/zernikeData.feather')
#fluxData_df


# 👩‍🍳 Prepare Dataset

In [ ]:
# Prepare the training dataset
X = fluxData_df
y = zernikeData_df
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)


# Build simple Model

In [ ]:
#AoModel = keras.Sequential([
#    
#     keras.layers.InputLayer(19, name="digits"),
#     keras.layers.Dense(2000, activation="relu"),
#     keras.layers.Dense(1050, activation="relu"),
#     keras.layers.Dense(100, activation="relu"),
#     keras.layers.Dense(9, name="predictions"),
#
#])
#AoModel.summary()

## Config parameter

In [ ]:
run = wandb.init(project="Ao_Paper_Model",
notes="Learning rate scheduler",
tags=['Ao', "Paper", "lr_schedule" ,"1DConv"])

In [ ]:



config = run.config




config.conv1D_1 = 64
config.conv1D_1_width = 4
config.conv1D_2 = 32
config.conv1D_2_width = 2

In [ ]:
 input_shape = (4, 19, 1)
AoModel = keras.Sequential([
     
     
     #keras.layers.Conv1D(32, 3, activation='relu',input_shape=(None,19), name="conv1d"),
     keras.layers.Conv1D( config.conv1D_1, config.conv1D_1_width, activation='relu',input_shape=input_shape[1:]),
     keras.layers.MaxPool1D(pool_size=2),
     #keras.layers.Dropout(rate = 0.2),
     keras.layers.Conv1D( config.conv1D_2, config.conv1D_2_width, activation='relu',input_shape=input_shape[1:]),
     keras.layers.MaxPool1D(pool_size=2),


     #keras.layers.Dropout(rate = 0.2),
     keras.layers.Flatten(),
     keras.layers.Dense(2000, activation="relu"),
     #keras.layers.Dropout(rate = 0.2),
     keras.layers.Dense(1050, activation="relu"),
     #keras.layers.Dropout(rate = 0.1),

     keras.layers.Dense(100, activation="relu"),
     #keras.layers.Dropout(rate = 0.2),

     keras.layers.Dense(9,  name="predictions"),

])
AoModel.summary()

## Reshape training data

In [ ]:
input_train = x_train.to_numpy()[:,:,np.newaxis]
input_test = x_test.to_numpy()[:,:,np.newaxis]

Train simple model

In [ ]:
config.batch_size = 128
config.epochs = 20
config.loss = "mse"
config.initalLearningRate = 5e-2

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=config.initalLearningRate,
    decay_steps=1000,
    decay_rate=0.96)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
AoModel.compile(optimizer= "adam", loss=config.loss,
    metrics=[keras.metrics.RootMeanSquaredError()])

AoModel.fit(input_train, y_train, batch_size=config.batch_size, epochs=config.epochs, validation_split=0.1, callbacks=[WandbCallback()])



In [ ]:
trainPredict = AoModel.predict(input_train)
trainError = trainPredict - y_train
tmp = trainError.to_numpy().flatten()
np.sqrt(np.sum(tmp**2)/tmp.size)/math.pi/.0051

In [ ]:
trainPredict = AoModel.predict(input_test)
trainError = trainPredict - y_test
tmp = trainError.to_numpy().flatten()
np.sqrt(np.sum(tmp**2)/tmp.size)/math.pi/.0051

In [ ]:
np.sqrt(np.sum(tmp**2)/tmp.size)